### Load Packages and Data

In [ ]:
### time series forecasting 
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from numpy.typing import ArrayLike

from src import config

pd.set_option("display.max_columns", None)

%matplotlib inline
sns.set_style('white')
sns.set_palette('deep')

In [ ]:
!pip install statsmodels
!pip install xgboost

In [ ]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.api import ExponentialSmoothing

In [ ]:
# read in data 
df = pd.read_parquet(config.INT_FILE_PATH / 'transactions.parquet')

# preview the data
df.head()

In [ ]:
# select only necessary rows
df = df.loc[:, ["order_purchase_timestamp", "order_total_price"]]

In [ ]:
# resample to weekly time period
weekly_sales = df.set_index("order_purchase_timestamp").resample("W")[["order_total_price"]].sum()

In [ ]:
# preview the data
weekly_sales

### Decomposition

In [ ]:
decompose = seasonal_decompose(weekly_sales, extrapolate_trend=12)

In [ ]:
# observed
obs = decompose.observed
# trend
trend = decompose.trend
# seasonal
season = decompose.seasonal
# error
random = decompose.resid

In [ ]:
fig, axes = plt.subplots(4, 1, figsize=(15,8), sharex=True)
fig.suptitle('Time Series of Purchase Values')

sns.lineplot(x=obs.index, y=obs, ax=axes[0], data=obs)
sns.lineplot(x=trend.index, y=trend, ax=axes[1], data=trend)
sns.lineplot(x=season.index, y=season, ax=axes[2], data=season)
sns.lineplot(x=random.index, y=random, ax=axes[3], data=random)

plt.show()

In [ ]:
weekly_sales.info()

In [ ]:
# remove the last 8 weeks with erroneous 0 values
weekly_sales_raw = weekly_sales.iloc[:-8, :]

In [ ]:
### Replace 0 values with a small value
weekly_sales_clean = weekly_sales_raw.copy(deep=True)
weekly_sales_clean.loc[weekly_sales_clean["order_total_price"] == 0, "order_total_price"] = 1

### Train test split

In [ ]:
train_size = int(len(weekly_sales_clean) * 0.80)
train, test = weekly_sales_clean[:train_size], weekly_sales_clean[train_size:]

In [ ]:
train.head()

In [ ]:
# X_train, y_train = train.drop(columns="order_total_price"), train["order_total_price"]
# X_test, y_test = test.drop(columns="order_total_price"), test["order_total_price"]

In [ ]:
## create function to define MAPE
def mean_absolute_percentage_error(actual: ArrayLike, pred: ArrayLike): 
    actual, pred = np.array(actual), np.array(pred)
    return np.mean(np.abs((actual - pred) / actual))

### Triple Exponential Smoothing Models

In [ ]:
# additive trend and seasonal
HWM1 = ExponentialSmoothing(
    train,
    seasonal_periods=12,
    trend="add",
    seasonal="add",
    use_boxcox=True,
    initialization_method="estimated",
).fit()

fcastHWM1 = HWM1.forecast(21).rename("Holt's Winters additive trend")

plt.figure(figsize=(12,8))
plt.xticks(rotation=45)
plt.title('Order Total Price')
plt.plot(weekly_sales_clean['order_total_price'], marker="o", color="black")
plt.plot(HWM1.fittedvalues, marker="o", color="blue")
(line1,) = plt.plot(fcastHWM1, marker="o", color="blue")
plt.legend([line1],[fcastHWM1.name])
plt.show()

In [ ]:
# additive trend and multiplicative seasonal
HWM2 = ExponentialSmoothing(
    train,
    seasonal_periods=24,
    trend="add",
    seasonal="mul",
    use_boxcox=True,
    initialization_method="estimated",
).fit()

fcastHWM2 = HWM2.forecast(21).rename("Holt's Winters additive trend")

plt.figure(figsize=(12,8))
plt.xticks(rotation=45)
plt.title('Order Total Price')
plt.plot(weekly_sales_clean['order_total_price'], marker="o", color="black")
plt.plot(HWM2.fittedvalues, marker="o", color="blue")
(line2,) = plt.plot(fcastHWM2, marker="o", color="blue")
plt.legend([line2],[fcastHWM2.name])
plt.show()

In [ ]:
# additive model with damped trend
HWM3 = ExponentialSmoothing(
    train,
    seasonal_periods=12,
    trend="add",
    seasonal="add",
    damped_trend=True,
    use_boxcox=True,
    initialization_method="estimated",
).fit()

fcastHWM3 = HWM3.forecast(50).rename("Holt's Winters additive damped trend")

plt.figure(figsize=(12,8))
plt.xticks(rotation=45)
plt.title('Order Total Price')
plt.plot(weekly_sales_clean['order_total_price'], marker="o", color="black")
plt.plot(HWM3.fittedvalues, marker="o", color="blue")
(line3,) = plt.plot(fcastHWM3, marker="o", color="blue")
plt.legend([line3],[fcastHWM3.name])
plt.show()

In [ ]:
# multiplicative model with damped trend
HWM4 = ExponentialSmoothing(
    train,
    seasonal_periods=12,
    trend="add",
    seasonal="mul",
    damped_trend=True,
    use_boxcox=True,
    initialization_method="estimated",
).fit()

fcastHWM4 = HWM4.forecast(21).rename("Holt's Winters multiplicative damped trend")

plt.figure(figsize=(12,8))
plt.xticks(rotation=45)
plt.title('Order Total Price')
plt.plot(weekly_sales_clean['order_total_price'], marker="o", color="black")
plt.plot(HWM4.fittedvalues, marker="o", color="blue")
(line4,) = plt.plot(fcastHWM4, marker="o", color="blue")
plt.legend([line4],[fcastHWM4.name])
plt.show()

### Evaluation 

In [ ]:
# generate forecasts for each model
fcastHWM1 = HWM1.forecast(len(test)).rename("Holt's Winters additive trend")
fcastHWM2 = HWM2.forecast(len(test)).rename("Holt's Winters multiplicative trend")
fcastHWM3 = HWM3.forecast(len(test)).rename("Holt's Winters additive damped trend")
fcastHWM4 = HWM4.forecast(len(test)).rename("Holt's Winters multiplicative damped trend")

# Find lowest MAPE from the 4 HWM models
min_mape = np.inf
min_HWM = pd.DataFrame()
for HWM in [fcastHWM1, fcastHWM2, fcastHWM3, fcastHWM4]:
    cur_mape = mean_absolute_percentage_error(test, HWM)
    if cur_mape < min_mape:
        min_mape = cur_mape
        min_HWM = pd.DataFrame(HWM)
        min_model = HWM.name

# return best model and MAPE
print(f"Best Model: {min_model}")
print(f"MAPE: {min_mape:.2%}")

### Ensemble Based Models 

In [ ]:
# preview data
weekly_sales_clean.head()

### Preprocessing

In [ ]:
# create 1, 2, 3 week lag periods
weekly_sales_clean["X_1"] = weekly_sales_clean["order_total_price"].shift(1)
weekly_sales_clean["X_2"] = weekly_sales_clean["order_total_price"].shift(2)
weekly_sales_clean["X_3"] = weekly_sales_clean["order_total_price"].shift(3)

In [ ]:
# split models and prepare X and y arrays
train_size = int(len(weekly_sales_clean) * 0.8)
train_ensemble, test_ensemble = weekly_sales_clean[:train_size], weekly_sales_clean[train_size:]

In [ ]:
X_train, y_train = train_ensemble.drop(columns="order_total_price"), train_ensemble["order_total_price"]
X_test, y_test = test_ensemble.drop(columns="order_total_price"), test_ensemble["order_total_price"]

### Modelling

In [ ]:
xgb_model = xgb.XGBRegressor(n_estimators=400, learning_rate=0.03).set_params(early_stopping_rounds=3)
xgb_model.fit(X_train, y_train, 
              eval_set=[(X_test, y_test)], 
              verbose=False)

y_pred = xgb_model.predict(X_test)

print(f"RMSE for xgb is: {mean_squared_error(y_test, y_pred, squared=False):.2f}")
print(f"MAPE for xgb is {mean_absolute_percentage_error(y_test, y_pred):.2%}")

In [ ]:
train_pred = pd.Series(xgb_model.predict(X_train))
test_pred = pd.Series(xgb_model.predict(X_test))

predictions = pd.concat([train_pred, test_pred], axis=0)
predictions.index = weekly_sales_filt.index

ax = plt.gca()

weekly_sales_clean[['order_total_price']].plot(figsize=(15, 6), ax=ax, marker="o", color="black")
predictions.plot(figsize=(15, 6), ax=ax, marker="o", color="blue")
plt.show()